In [17]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import NearestCentroid

In [18]:

def extract_hog_features(image, image_size=(128, 64)):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if image.shape[:2] != image_size[::-1]:
        image = cv2.resize(image, image_size, interpolation=cv2.INTER_LINEAR)
    hog = cv2.HOGDescriptor(
        (image_size[0], image_size[1]),  # winSize
        (8, 8),      # blockSize
        (8, 8),      # blockStride
        (4, 4),      # cellSize
        9           # nbins
    )
    features = hog.compute(image)
    
    return features.ravel() 

In [19]:
def load_hog_features_from_folders(base_folder, subfolders, specific_folder, image_size=(128, 64)):
    hog_features = []
    labels = []
    for subject in os.listdir(base_folder):
        for subfolder in subfolders:
            folder_path = os.path.join(base_folder, subject, subfolder, specific_folder)
            if os.path.isdir(folder_path):
                for img_name in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, img_name)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        hog_features.append(extract_hog_features(img, image_size))
                        labels.append(subject)
    return np.array(hog_features), np.array(labels)

In [20]:
base_folder = '/home/sazid/Model_Based_GAIT_PROJECT/Pre_Processing/output/MFEI_Image/'
train_folders = ['nm-01', 'nm-02', 'nm-03', 'nm-04','nm-05', 'nm-06', 'cl-01', 'cl-02']
test_folders = ['bg-01', 'bg-02']
folders = ['000', '018', '036', '054', '072', '090', '108', '126', '144', '162', '180']

In [21]:
output_dir = '/home/sazid/Model_Based_GAIT_PROJECT/Pre_Processing/output/Excelfile'
os.makedirs(output_dir, exist_ok=True)

In [22]:
results = []

In [23]:
for train_folder in folders:
    for test_folder in folders:
        X_train, y_train = load_hog_features_from_folders(base_folder, train_folders, train_folder)
        X_test, y_test = load_hog_features_from_folders(base_folder, test_folders, test_folder)
        
        lda = LDA(n_components=min(len(np.unique(y_train)) - 1, X_train.shape[1]))
        X_train_lda = lda.fit_transform(X_train, y_train)
        X_test_lda = lda.transform(X_test)
        
        ncc = NearestCentroid()
        ncc.fit(X_train_lda, y_train)
        y_pred_ncc = ncc.predict(X_test_lda)
        
        results.append({
            'Classifier': 'Nearest Centroid',
            'Train Folder': train_folder,
            'Test Folder': test_folder,
            'Accuracy': accuracy_score(y_test, y_pred_ncc),
            'Precision': precision_score(y_test, y_pred_ncc, average='macro'),
            'Recall': recall_score(y_test, y_pred_ncc, average='macro'),
            'F1-Score': f1_score(y_test, y_pred_ncc, average='macro')
        })

/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [24]:
results_df = pd.DataFrame(results)
output_file_path = os.path.join(output_dir, 'nc_bg01_02_test_mfei.xlsx')
results_df.to_excel(output_file_path, index=False)
print(f"Results saved to: {output_file_path}")

Results saved to: /home/sazid/Model_Based_GAIT_PROJECT/Pre_Processing/output/Excelfile/nc_bg01_02_test_mfei.xlsx
